In [31]:
class Byte:
    def __init__(self, x=0):
        self.val = x & 0xff

    def update(self):
        self.val &= 0xff

    def get(self):
        self.update()
        return self.val

class Bit32:
    def __init__(self, x=0):
        self.val = x & 0xffffffff

    def update(self):
        self.val = self.val & 0xffffffff

    def get(self):
        self.update()
        return self.val

    def __setitem__(self, index, value):
        if value:
            self.val = self.val | (value << index)
        else:
            self.val = self.val & ~(value << index)
        self.update()

    def __getitem__(self, index):
        self.update()
        if self.val & (1 << index):
            return 1
        else:
            return 0

class Registers:
    def __init__(self, n):
        self.files = [Bit32() for _ in range(n)]
        self.size = n

    def __getitem__(self, item):
        return self.files[item]

    def __setitem__(self, key, value):
        self.files[key] = value

    def output(self):
        for i in range(self.size):
            print("$%d:\t0x%016x\t%d" % (i, self.files[i].val, self.files[i].val))
            


class Registers_file:
    def __init__(self, n=32):
        self.files = Registers(32)

    def ports_out(self, a_addr, b_addr):
        a_out = Bit32(0) if not a_addr else self.files[a_addr]
        b_out = Bit32(0) if not b_addr else self.files[b_addr]
        return a_out, b_out

    def port_single_out(self, addr) -> Bit32:
        return Bit32(0) if not addr else self.files[addr]

    def ports_in(self, addr, c_in: Bit32):
        if addr:
            self.files[addr] = c_in

    def output(self):
        self.files.output()
        

In [32]:
def byte_to_bit16(byte1, byte0):
    return (byte1 << 8) | byte0

def bit16_to_byte(bits):
    byte1 = (bits >> 8) & 0xff
    byte0 = bits & 0xff
    return byte1, byte0


def byte_to_bit32(byte3, byte2, byte1, byte0):
    bit16_1 = byte_to_bit16(byte3, byte2)
    bit16_0 = byte_to_bit16(byte1, byte0)
    return (bit16_1 << 16) | bit16_0


def bit32_to_byte(bits):
    bit16_1 = (bits >> 16) & 0xffff
    bit16_0 = bits & 0xffff
    byte3, byte2 = bit16_to_byte(bit16_1)
    byte1, byte0 = bit16_to_byte(bit16_0)
    return byte3, byte2, byte1, byte0

class Memory:
    def __init__(self, size):
        self.files = [Byte() for _ in range(size)]
        self.size = size

    def data_32_port_out(self, addr) -> Bit32:
        byte0, byte1, byte2, byte3 = self.files[addr].val, self.files[addr + 1].val, \
                                     self.files[addr + 2].val, self.files[addr + 3].val
        return Bit32(byte_to_bit32(byte3, byte2, byte1, byte0))

    def data_32_port_in(self, bits: Bit32, addr):
        byte3, byte2, byte1, byte0 = bit32_to_byte(bits.get())
        self.files[addr] = Byte(byte0)
        self.files[addr + 1] = Byte(byte1)
        self.files[addr + 2] = Byte(byte2)
        self.files[addr + 3] = Byte(byte3)
    
    def force_set(self, byte: Byte, addr: int):
        self.files[addr] = byte
        
    def force_set_seq(self, b: list, addr):
        for i in range(len(b)):
            self.files[addr + i] = Byte(b[i])

    def output(self):
        for i in range(0x1000, 0x1000 + 64, 4):
            print("0x%08x:" % i, end=" ")
            for j in range(4):
                print("%02x" % self.files[i + j].val, end="\t")
            print("")
            
    def output_range(self, low, high):
        for i in range(low, high, 4):
            print("0x%08x:" % i, end=" ")
            for j in range(4):
                print("%02x" % self.files[i + j].val, end="\t")
            print("")


In [33]:
def sign_expend(x, len):
    if x & (1 << len - 1):
        return x - (1 << len)
    else:
        return x


def mask(n):
    return (1 << n) - 1


def get_bit(x, L, R):
    return x >> R & mask(L - R + 1)


class type:
    def __init__(self):
        self.name = ""
        self.do = 0  # 0 -> reg, 1 -> save, 2 -> load, 3 -> branch
        self.imm = False
        self.wb = False

I = (0b0010011, 0b0000011, 0b1100111, 0b1110011, 0b0000111, 0b0011011)

def decoder_32(instr):
    job = type()
    rs1 = 0
    rs2 = 0
    rd = 0
    imm = 0
    job.imm = False
    opcode = instr & 0b1111111
    if opcode == 0b0110011:  # R-type
        funct7 = get_bit(instr, 31, 25)
        funct3 = get_bit(instr, 14, 12)
        rs2 = get_bit(instr, 24, 20)
        rs1 = get_bit(instr, 19, 15)
        rd = get_bit(instr, 11, 7)
        if funct3 == 0b000:
            if funct7 == 0x00:
                job.name = "add"
            elif funct7 == 0x01:
                job.name = "mul"
            elif funct7 == 0x20:
                job.name = "sub"

    elif opcode in I:  # I-type
        imm_pre = (get_bit(instr, 31, 20))
        imm = Bit32(sign_expend(imm_pre, 12))
        rs1 = get_bit(instr, 19, 15)
        funct3 = get_bit(instr, 14, 12)
        rd = get_bit(instr, 11, 7)
        job.imm = True
        if opcode == 0b0010011:
            if funct3 == 0b000:
                job.name = "add"  # addi
        elif opcode == 0b0000011:
            if funct3 == 0b010:
                job.name = "lw"
                job.do = 2
        elif opcode == 0b1100111:
            job.name = "jalr"
            job.do = 3
    elif opcode == 0b0110111:  # U-type
        job.name = "lui"
        job.imm = True
        job.do = 4
        rd = get_bit(instr, 11, 7)
        imm = Bit32(get_bit(instr, 31, 12) << 12)
    elif opcode == 0b1101111:  # J-type
        job.name = "jal"
        job.imm = True
        job.do = 3
        idx_3112 = get_bit(instr, 31, 12)
        imm_pre = (get_bit(idx_3112, 19, 19) << 20) | (get_bit(idx_3112, 7, 0) << 12) | (
                    get_bit(idx_3112, 8, 8) << 11) | (get_bit(idx_3112, 18, 9) << 1)
        imm = Bit32(sign_expend(imm_pre, 21))
        rd = get_bit(instr, 11, 7)
    elif opcode == 0b0100011:  # S-type
        imm_pre = (get_bit(instr, 31, 25) << 5) | get_bit(instr, 11, 7)
        imm = Bit32(sign_expend(imm_pre, 12))
        job.imm = True
        rs2 = get_bit(instr, 24, 20)
        rs1 = get_bit(instr, 19, 15)
        funct3 = get_bit(instr, 14, 12)
        if funct3 == 0b010:
            job.name = "sw"
            job.do = 1
            # job.size = 1
    elif opcode == 0b1100011:  # B-type
        rs2 = get_bit(instr, 24, 20)
        rs1 = get_bit(instr, 19, 15)
        funct3 = get_bit(instr, 14, 12)
        idx_3125 = get_bit(instr, 31, 25)
        idx_117 = get_bit(instr, 11, 7)
        imm_pre = (get_bit(idx_3125, 6, 6) << 12) | (get_bit(idx_117, 0, 0) << 11) | (get_bit(idx_3125, 5, 0) << 5) | (
                    get_bit(idx_117, 4, 1) << 1)
        imm = Bit32(sign_expend(imm_pre, 13))
        job.imm = True
        if funct3 == 0b000:
            job.name = "beq"
            job.do = 3
        elif funct3 == 0b001:
            job.name = "bne"
            job.do = 3
        elif funct3 == 0b100:
            job.name = "blt"
            job.do = 3
        elif funct3 == 0b101:
            job.name = "bge"
            job.do = 3
        elif funct3 == 0b110:
            job.name = "blt"  # bltu
            job.do = 3
        elif funct3 == 0b111:
            job.name = "bge"  # bgeu
            job.do = 3

    return job, rs1, rs2, rd, imm

In [34]:
def execute(rs1: Bit32, rs2: Bit32, imm: Bit32, pc: Bit32, job: type):
    dst = Bit32(0)
    addr = Bit32(0)
    pc_next = Bit32(pc.val + 4)
    content = Bit32(0)
    if job.do == 0:
        job.wb = not job.wb
        src1 = rs1
        if job.imm:
            src2 = imm
        else:
            src2 = rs2
        if(job.name == "add"):
            dst = Bit32(sign_expend(Bit32(src1.val + src2.val).val & 0xffffffff, 32))
    
    elif job.do == 1 or job.do == 2:
        addr = Bit32(rs1.val + imm.val)
        if job.do == 1:
            content = rs2
        else:
            job.wb = True
    
    elif job.do == 3:
        if job.name[0] == "j":
            dst = Bit32(pc.val + 4)
            job.wb = True
            if job.name == "jal":
                pc_next = Bit32(pc.val + imm.val)
        else:
            pc_next_tmp = Bit32(pc.val + imm.val)
            if job.name == "blt" and sign_expend(rs1.val, 32) < sign_expend(rs2.val, 32):
                pc_next = pc_next_tmp
    return dst, addr, pc_next, content
            
    
        

In [35]:
rf = Registers_file()
# 地址空间太大，仅模拟0 ~ 1024的地址
mem = Memory(1024)

#   lw x1, 0x100(x0) `10002083
#   blt x1, x0, else `0000c663
# 	addi x1, x1, 1   `00108093
#   j then           `00c0006f
# else:
# 	lw x2, 0x104(x0) `10402103
# 	add x1, x1, x2   `002080b3
# then:
# 	sw x1, 0x108(x0) `10102423

instr = [0x83, 0x20, 0x00, 0x10,
         0x63, 0xc6, 0x00, 0x00,
         0x93, 0x80, 0x10, 0x00,
         0x6f, 0x00, 0xc0, 0x00,
         0x03, 0x21, 0x40, 0x10,
         0xb3, 0x80, 0x20, 0x00,
         0x23, 0x24, 0x10, 0x10]

data = [0xff, 0xff, 0xff, 0xff,
        0x02, 0x00, 0x00, 0x00]


mem.force_set_seq(instr, 0)
mem.force_set_seq(data, 0x100)
# mem.output_range(0, 28)
pc = Bit32(0)
while True:
    # print(pc.val)
    if pc.val > 28:
        break
    instr = mem.data_32_port_out(pc.val)
    # pc = pc + 4
    job, rs1, rs2, rd, imm = decoder_32(instr.val)
    rs1_r, rs2_r = rf.ports_out(rs1, rs2)
    dst, addr, pc, content = execute(rs1_r, rs2_r, imm, pc, job)
    
    if job.do == 1: #save
        mem.data_32_port_in(content, addr.val)
    elif job.do == 2: #load
        dst = mem.data_32_port_out(addr.val)
    if job.wb:
        rf.ports_in(rd, dst)
        
mem.output_range(0x100, 0x10c)


0x00000100: ff	ff	ff	ff	
0x00000104: 02	00	00	00	
0x00000108: 01	00	00	00	
